In [ ]:
# REFERENCES:
# https://towardsdatascience.com/extracting-text-from-scanned-pdf-using-pytesseract-open-cv-cd670ee38052
# https://www.codegrepper.com/code-examples/python/pytesseract+pdf+to+text
# https://www.google.com/search?q=python+pytesseract+pdf+to+text&oq=python+pdf+pytesse&aqs=edge.2.69i57j0i22i30l2.9015j0j1&sourceid=chrome&ie=UTF-8

#https://stackoverflow.com/questions/67516273/remove-header-and-footer-from-pdftotext-module-in-python
#https://www.anycodings.com/1questions/155089/remove-header-and-footer-from-pdftotext-module-in-python
#https://pyimagesearch.com/2015/02/09/removing-contours-image-using-python-opencv/
#https://pdf.wondershare.com.br/pdf-knowledge/pdf-to-png-python.html

In [ ]:
# INSTALATIONS, if necessary:
#!pip install opencv-python   # cv2
#!pip install pytesseract
#!pip install pdf2image
#!pip install poppler-utils
#!apt-get install poppler-utils
#!apt-get update
#!apt-get install libleptonica-dev tesseract-ocr python3-pil tesseract-ocr-por tesseract-ocr-eng tesseract-ocr-script-latn

In [ ]:
# IMPORTS:
import cv2
import pytesseract
from PIL import Image
from pdf2image import convert_from_path

In [ ]:
# VARIABLES:
custom_config = r'--oem 3 --psm 11'
file_path = "/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/"   #file_path = "/content/"
output_folder="/home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs"  #arq = file_path+"00002067920105010076-29-03-2016.pdf"
arq = "0080300-68.2008.5.01.0016.pdf"
path_arq = file_path+arq
path_arq

In [ ]:
# TESTS:

In [ ]:
pages = convert_from_path(arq, 500, output_folder=output_folder, first_page=1, last_page=1, fmt='jpg')

images = []

for i in range(len(pages)):
    page_name = arq+"_page_"+str(i)+".jpg"
    pages[i].save(page_name, "JPEG")
    images.append(page_name)


In [ ]:
for i in range(len(images)):
    text = pytesseract.image_to_string(images[i], lang='por')
    print(text)

In [ ]:
for i in range(len(images)):
    img = cv2.imread(images[i])
    
#gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#blur = cv2.GaussianBlur(gray, (9,9), 0)
#thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,30)
#text = pytesseract.image_to_string(blur, lang="por")
#text.replace("\t","")
#text.replace("\n","")
#print(text)

In [196]:
%%writefile HelperFunctions/DataWrangling/criaCorpusWithOCR.py

# 1.Extração de texto dos pdf das sentenças baixas
#
# Cria Corpus (DatasetClassification.CSV) das Sentenças do TRT1
# Le os arquivos .txt e armazena-os em uma lista e depois persiste-o em disco no formato .csv.
#
import io
import pathlib
import pandas as pd
import datasets
from datasets import load_dataset
from datasets import Dataset
from os import listdir
from os.path import isfile, join
import re
# IMPORTS:
import cv2
import pytesseract
from PIL import Image, ImageOps
from pdf2image import convert_from_path
import numpy
    
class Corpus():
    
    def __init__(self):
        # Agrupar estas variaveis em um arquivo para carregar nas classes. VARIABLES:
        self.custom_config = r'--oem 1 --psm 4'
        self.file_path = "/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/"   #file_path = "/content/"
        self.output_folder="/home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs/"  #arq = file_path+"00002067920105010076-29-03-2016.pdf"
        self.arq = "0080300-68.2008.5.01.0016.pdf"
        self.path_arq = self.file_path+self.arq
        self.images = []
        self.textos = []
        self.pages = []

        
        
    def pdftoimg(self):
        
        self.pages = convert_from_path(self.path_arq, 500, output_folder=self.output_folder+"imgs/", fmt='jpg') # first_page=1, last_page=1, 

        for i, page in enumerate(self.pages):
            page_name = self.output_folder+self.arq+"_page_"+str(i)+".jpg"
            page.save(page_name, "JPEG")
            self.images.append(page_name)

            
    
    def crop_img(self):
        for i, page in enumerate(self.pages):
                        
            # open method used to open different extension image file
            # pil_image = Image.open(self.output_folder+self.arq+"_page_"+str(i)+".jpg") 
            #width, height = pil_image.size
            # gray_img    = ImageOps.grayscale(pil_image)
            # bk_wh_img   = gray_img.convert('1', dither=Image.NONE) # convert image to black and white
            
            img = cv2.imread(self.output_folder+self.arq+"_page_"+str(i)+".jpg", cv2.IMREAD_GRAYSCALE)
            # You may need to convert the color.
            #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            (thresh, im_bw) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
            #dst = cv2.detailEnhance(im_bw, sigma_s=10, sigma_r=0.15)
            
            pil_image = Image.fromarray(im_bw)

            frac   = 0.85
            left   = pil_image.size[0]*((1-frac)/2)
            upper  = pil_image.size[1]*((1-frac)/2)
            right  = pil_image.size[0]-((1-frac)/2)*pil_image.size[0]
            bottom = pil_image.size[1]-((1-frac)/2)*pil_image.size[1]
        
            cropped_img = pil_image.crop((left*1.3, upper*1.8, right, bottom*2.5))           
            cropped_img.save(self.output_folder+self.arq+"_page_"+str(i)+".jpg")      

         
    
    def imgtotext(self):
        
        for i, page in enumerate(self.pages):
            # Recognize the text as string in image using pytesserct 
            #pytesseract.pytesseract.tesseract_cmd = tesseract_path
            #text = str(((pytesseract.image_to_string(Image.open(img))))) 
            text = str(((pytesseract.image_to_string(self.output_folder+self.arq+"_page_"+str(i)+".jpg", lang='por', config=self.custom_config)))) 
            text.replace("\n", " ")
            text.replace('-\n', ' ')
            self.textos.append(text)

        print(len(self.images))
        res = " ".join(self.textos)
        return res

    
    
    def proccess_folder(self):
        dataFile = {}

        directoryList = os.listdir(self.file_path)
        for pdfFile in directoryList:
            if pdfFile.endswith('.pdf'):
                self.arq = pdfFile
                self.path_arq = file_path+arq
                self.pages = []
                pdftoimg()
                #crop_img()
                imgtotext()
                
                pages = convert_from_path(folder+'/'+pdfFile)
                for page in pages:
                    text = pytesseract.image_to_string(page, config=config)
                    dataFile[pdfFile.split('/')[-1].split('.')[0]+'_'+str(pages.index(page)+1)] = text
        return dataFile

Overwriting HelperFunctions/DataWrangling/criaCorpusWithOCR.py


In [197]:
# 2. Criação do objeto
from HelperFunctions.DataWrangling.criaCorpusWithOCR import Corpus
%run -i HelperFunctions/DataWrangling/criaCorpusWithOCR.py
corpus = Corpus()

In [198]:
%%time
corpus.pdftoimg()

CPU times: user 680 ms, sys: 203 ms, total: 883 ms
Wall time: 3.61 s


In [199]:
%%time
corpus.crop_img()

CPU times: user 911 ms, sys: 242 ms, total: 1.15 s
Wall time: 811 ms


In [200]:
%%time
print(corpus.imgtotext())
# psm 11

6
16º VARA DO TRABALHO DO RIO DE JANEIRO

ATA DE AUDIÊNCIA
Processo no. 803/08-0

Aos à & dias do mês de DE 3EmRpcdo ano de 2008, às 9'v O horas,
na sala de audiência desta Vara do Trabalho, na presença da Dra. ANA CRISTINA
MAGALHÃES FONTES, MMa. Juiza do Trabalho, foram apregoados os litigantes:
JUAREZ JOSE DA SILVA, reclamante e GASTROSERVICE REFEIÇÕES LTDA,
reclamada.

Ausentes as partes.

Preenchidas as formalidades legais, nos termos do artigo 850 parágrafo único da
CLT, a Magistrada passou a proferir a seguinte decisão.

SENTENÇA

|- RELATÓRIO

JUAREZ JOSE:DA SILVA, reclamante e GASTROSERVICE REFEIÇÕES LTDA,
reclamada litigam neste processo, no qual o primeiro ajuizou ação trabalhista em face do
segundo, alegando fatos e fundamentos e postulando os títulos discriminados na inicial.

Devidamente notificada, compareceu a reclamada à sessão de audiência,
defendendo-se com as razões de fis. 46/62, acompanhada de documentos, argúindo a
preliminar de inépcia e prejudicial de mérito de 

In [166]:
%%time
print(corpus.imgtotext())
# psm 11

6
0

PODER JUDICIÁRIO FEDERAL |
JUSTIÇA DO TRABALHO S
TRIBUNAL REGIONAL DO TRABALHO DA 1º REGIÃO

 

16º VARA DO TRABALHO DO RIO DE JANEIRO

ATA DE AUDIÊNCIA
Processo no. 803/08-0

Aos A%& dias do mês de De 2Emapcdo ano de 2008, às 3-v O horas,
na sala de audiência desta Vara do Trabalho, na presença da Dra. ANA CRISTINA
MAGALHÃES FONTES, MMa. Juíza do Trabalho, foram apregoados os litigantes:
JUAREZ JOSE DA SILVA, reclamante e GASTROSERVICE REFEIÇÕES LTDA,
reclamada. '

Ausentes as partes.

Preenchidas as formalidades legais, nos termos do artigo 850 parágrafo único da
CLT, a Magistrada passou a proferir a seguinte decisão.

SE NTENC.A

|- RELATÓRIO

JUAREZ JOSE“DA SILVA, reclamante e GASTROSERVICE REFEIÇÕES LTDA,
reclamada litigam neste processo, no qual o primeiro ajuizou ação trabalhista em face do
segundo, alegando fatos e fundamentos e postulando os títulos discriminados na inicial.

Devidamente notificada, compareceu a reclamada à sessão de audiência,
defendendo-se com as razões

In [152]:
%%time
print(corpus.imgtotext())
# psm 11

6
16º VARA DO TRABALHO DO RIO DE JANEIRO

ATA DE AUDIÊNCIA
Processo no. 803/08-0

Aos h % dias do mês de De 3EmBecdo ano de 2008, às 9-v O horas,
na sala de audiência desta Vara do Trabalho, na presença da Dra. ANA CRISTINA
MAGALHÃES FONTES, MMa. Juíza do Trabalho, foram apregoados os litigantes:
JUAREZ JOSE DA SILVA, reclamante e GASTROSERVICE REFEIÇÕES LTDA,
reclamada.

Ausentes as partes.

Preenchidas as formalidades legais, nos termos do artigo 850 parágrafo único da
CLT, a Magistrada passou a proferir a seguinte decisão.

SE NA EN

| - RELATÓRIO

JUAREZ JOSE-DA SILVA, reclamante e GASTROSERVICE REFEIÇÕES LTDA,
reclamada litigam neste processo, no qual o primeiro ajuizou ação trabalhista em face do
segundo, alegando fatos e fundamentos e postulando os títulos discriminados na inicial.

Devidamente notificada, compareceu a reclamada à sessão de audiência,
defendendo-se com as razões de fis. 46/62, acompanhada de documentos, argúindo a
preliminar de inépcia e prejudicial de mérito de

In [16]:
%%time
print(corpus.imgtotext())
# psm 4

6
0X

PODER JUDICIÁRIO FEDERAL |
JUSTIÇA DO TRABALHO S
TRIBUNAL REGIONAL DO TRABALHO DA 1º REGIÃO |

 

16º VARA DO TRABALHO DO RIO DE JANEIRO

ATA DE AUDIÊNCIA
Processo no. 803/08-0

Aos a dias do mês de De 3Emegcdo ano de 2008, às o O horas,
na sala de audiência desta Vara do Trabalho, na presença da Dra. ANA CRISTINA
MAGALHÃES FONTES, MMa. Juíza do Trabalho, foram apregoados os litigantes:
JUAREZ JOSE DA SILVA, reclamante e GASTROSERVICE REFEIÇÕES LTDA,
reclamada.

Ausentes as partes.

Preenchidas as formalidades legais, nos termos do artigo 850 parágrafo único da
CLT, a Magistrada passou a proferir a seguinte decisão.

SENTENÇA

|- RELATÓRIO

JUAREZ JOSE'DA SILVA, reclamante e GASTROSERVICE REFEIÇÕES LTDA,
reclamada litigam neste processo, no qual o primeiro ajuizou ação trabalhista em face do
segundo, alegando fatos e fundamentos e postulando os títulos discriminados na inicial.

Devidamente notificada, compareceu a reclamada à sessão de audiência,
defendendo-se com as razões de f

In [28]:
%%time
print(corpus.imgtotext())
# --oem 1 --psm 4

6
0X

PODER JUDICIÁRIO FEDERAL |
JUSTIÇA DO TRABALHO S
TRIBUNAL REGIONAL DO TRABALHO DA 1º REGIÃO |

 

16º VARA DO TRABALHO DO RIO DE JANEIRO

ATA DE AUDIÊNCIA
Processo no. 803/08-0

Aos a dias do mês de De 3Emegcdo ano de 2008, às o O horas,
na sala de audiência desta Vara do Trabalho, na presença da Dra. ANA CRISTINA
MAGALHÃES FONTES, MMa. Juíza do Trabalho, foram apregoados os litigantes:
JUAREZ JOSE DA SILVA, reclamante e GASTROSERVICE REFEIÇÕES LTDA,
reclamada.

Ausentes as partes.

Preenchidas as formalidades legais, nos termos do artigo 850 parágrafo único da
CLT, a Magistrada passou a proferir a seguinte decisão.

SENTENÇA

|- RELATÓRIO

JUAREZ JOSE'DA SILVA, reclamante e GASTROSERVICE REFEIÇÕES LTDA,
reclamada litigam neste processo, no qual o primeiro ajuizou ação trabalhista em face do
segundo, alegando fatos e fundamentos e postulando os títulos discriminados na inicial.

Devidamente notificada, compareceu a reclamada à sessão de audiência,
defendendo-se com as razões de f

In [ ]:
%%time
print(corpus.imgtotext())
# psm


In [ ]:
%%time
print(corpus.imgtotext())
# psm

In [31]:
#!pip install ocrmypdf

In [33]:
import ocrmypdf
file = "/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/0080300-68.2008.5.01.0016.pdf"
result = ocrmypdf.ocr(file, 'OCR_'+file, output_type='pdf', skip_text=True, deskew=True)

OutputFileAccessError: Output file location (OCR_/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/0080300-68.2008.5.01.0016.pdf) is not a writable file.

In [37]:
!pip install PyMuPDF

In [41]:
import fitz

ImportError: cannot import name 'ParamSpec' from 'typing_extensions' (/home/info/.local/lib/python3.8/site-packages/typing_extensions.py)

In [42]:
file = "/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/0080300-68.2008.5.01.0016.pdf"
doc = fitz.open(file)

for page_num in range(doc.pageCount):
        page = doc.loadPage(page_num)
        pages_df = pages_df.append({'text': page.getText('text')}, ignore_index=True)
        
        
extraction_pdfs[file] = pages_df  

NameError: name 'fitz' is not defined

In [ ]:
for file in file_list:
    try:
        result = ocrmypdf.ocr(file, 'OCR_'+file,output_type='pdf',skip_text=True,deskew=True)
    except Exception as e:
        if hasattr(e,'message'):
            error_log[file] = e.message
        else:
            error_log[file] = e
        continue
        
'''

In [ ]:
# https://www.anycodings.com/1questions/155089/remove-header-and-footer-from-pdftotext-module-in-python